In [1]:
import os

In [2]:
'''
==============================================
** Defining the original (raw) dataset path **
==============================================
'''

DATASET_PATH = '../Dataset/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/'


In [3]:
print(len(os.listdir(DATASET_PATH)))

371


In [4]:
brain_directories = [f.path for f in os.scandir(DATASET_PATH) if f.is_dir()]

# brain id 355 has ill formatted segmented data - hence needs to be removed
brain_directories.remove(DATASET_PATH + 'BraTS20_Training_355')

print(len(brain_directories))

for dir_name in brain_directories:
    print(dir_name)
    


368
../Dataset/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_273
../Dataset/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_040
../Dataset/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_124
../Dataset/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_190
../Dataset/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_030
../Dataset/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_333
../Dataset/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_157
../Dataset/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_283
../Dataset/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_249
../Dataset/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_001
../Dataset/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/BraTS20_Training_097
../Dataset/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingDa

In [5]:
def getDataIds(directoryList):
    dataIds = []
    for i in range(0, len(directoryList)):
        dataIds.append(directoryList[i][directoryList[i].rfind('/') + 1 :] )
    return dataIds

brain_ids = getDataIds(brain_directories)
len(brain_ids), brain_ids[0]

(368, 'BraTS20_Training_273')

In [6]:
368 * 155

57040

In [7]:
# naming convention
###################

# 1_BraTS20_Training_273
# 1 representes -- image number 
# image numbering starts from 0
# total range - 0 to 154

# total brain_image_ids = 57040


In [8]:
brain_image_ids = []

for each_brain_id in brain_ids:
    for i in range(32, 128):
        image_id = str(i) + '_' + each_brain_id
        brain_image_ids.append(image_id)
        
# check if all the image ids are present
len(brain_image_ids), brain_image_ids[156], brain_image_ids[0]

(35328, '92_BraTS20_Training_040', '32_BraTS20_Training_273')

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
train_brain_image_ids, test_brain_image_ids = train_test_split(brain_image_ids, test_size = 0.2, shuffle=True, random_state = 25)

In [11]:
len(train_brain_image_ids) , len(test_brain_image_ids)

(28262, 7066)

In [12]:
train_brain_image_ids

['47_BraTS20_Training_262',
 '125_BraTS20_Training_317',
 '46_BraTS20_Training_170',
 '39_BraTS20_Training_288',
 '33_BraTS20_Training_081',
 '79_BraTS20_Training_133',
 '43_BraTS20_Training_039',
 '44_BraTS20_Training_038',
 '111_BraTS20_Training_305',
 '91_BraTS20_Training_218',
 '117_BraTS20_Training_096',
 '40_BraTS20_Training_065',
 '32_BraTS20_Training_317',
 '71_BraTS20_Training_041',
 '120_BraTS20_Training_357',
 '44_BraTS20_Training_111',
 '78_BraTS20_Training_269',
 '41_BraTS20_Training_244',
 '57_BraTS20_Training_052',
 '104_BraTS20_Training_221',
 '77_BraTS20_Training_316',
 '65_BraTS20_Training_199',
 '81_BraTS20_Training_027',
 '77_BraTS20_Training_157',
 '44_BraTS20_Training_208',
 '121_BraTS20_Training_196',
 '87_BraTS20_Training_213',
 '51_BraTS20_Training_014',
 '34_BraTS20_Training_315',
 '86_BraTS20_Training_362',
 '97_BraTS20_Training_008',
 '62_BraTS20_Training_276',
 '34_BraTS20_Training_097',
 '48_BraTS20_Training_293',
 '36_BraTS20_Training_276',
 '100_BraTS20_

In [13]:
import nibabel as nib
import itertools


In [14]:
# total number of classes - 4
# class labels = 0, 1, 2, 3
# 0 is common and present in all the segmentations

# we will create a dictionary for the train_image_ids and sort them according to their segmentations

In [15]:
numbers = [1, 2, 3]
combinations = []

for r in range(len(numbers)+1):
    for combination in itertools.combinations(set(numbers), r):
        combination = (0,) + combination
        combinations.append(combination)
        
print(combinations)

# train_dict = dict.fromkeys(combinations, list())
train_dict = dict()
for c in combinations:
    train_dict[c] = []

print(train_dict)


[(0,), (0, 1), (0, 2), (0, 3), (0, 1, 2), (0, 1, 3), (0, 2, 3), (0, 1, 2, 3)]
{(0,): [], (0, 1): [], (0, 2): [], (0, 3): [], (0, 1, 2): [], (0, 1, 3): [], (0, 2, 3): [], (0, 1, 2, 3): []}


In [16]:
import numpy as np

In [17]:
progress = 0
for each_image_id in train_brain_image_ids:
    progress += 1
    if(progress % 1000 == 0):
        print(progress)
    image_number = each_image_id[: each_image_id.find('_')]
    brain_id = each_image_id[each_image_id.find('_')+1:]
    seg_mask = nib.load(DATASET_PATH + f'{brain_id}/{ brain_id}_seg.nii').get_fdata()
    seg_mask[seg_mask==4] = 3
#     print(image_number, brain_id,  np.unique(seg_mask[:, :, int(image_number)]))
    seg_mask_classes = tuple(np.unique(seg_mask[:, :, int(image_number)]).astype(int))
    train_dict[seg_mask_classes].append(each_image_id)
    


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000


In [18]:
for key in train_dict.keys():
    total_images = len(train_dict[key])
    print(key, total_images)

(0,) 9663
(0, 1) 58
(0, 2) 5553
(0, 3) 39
(0, 1, 2) 1733
(0, 1, 3) 82
(0, 2, 3) 1141
(0, 1, 2, 3) 9993


In [19]:
k_fold_split = 5

In [20]:
k_folds = []

for i in range(k_fold_split):
    k_folds.append([])

for key in train_dict.keys():
    total_images = len(train_dict[key])
    
    division = total_images//k_fold_split
    print(key, total_images, division, end=" ")

    if total_images > k_fold_split:
        for i in range(k_fold_split):
            k_folds[i] = k_folds[i] + train_dict[key][i*division : (i+1)*division]
        print("done", end="")
            
    else:
        for i in range(total_images):
            k_folds[i].append(train_dict[key][i])
    print()
        

(0,) 9663 1932 done
(0, 1) 58 11 done
(0, 2) 5553 1110 done
(0, 3) 39 7 done
(0, 1, 2) 1733 346 done
(0, 1, 3) 82 16 done
(0, 2, 3) 1141 228 done
(0, 1, 2, 3) 9993 1998 done


In [28]:
# 5234 + 14 + 1226 + 9 + 366 + 20 + 231 + 2023

9123

In [21]:
1932 + 11 + 1110 + 7 + 346 + 16 + 228 + 1998

5648

In [22]:
for i in range(k_fold_split):
    print(len(k_folds[i]))

5648
5648
5648
5648
5648


In [23]:
import pickle 

In [24]:

with open('training_data_ids_dict_1.pkl', 'wb') as file:
      pickle.dump(train_dict, file)
        
with open('stratified_k_folds_1.pkl', 'wb') as file:
      pickle.dump(k_folds, file)
        
with open('testing_data_ids_1.pkl', 'wb') as file:
      pickle.dump(test_brain_image_ids, file)

In [25]:
# we are done with creating stratified k_folds (k=5) and testing data 

In [37]:
progress= 0
for i in range(k_fold_split):
    fold_data_X = []
    fold_data_Y = []
    print(len(k_folds[i]))
    for each_image_id in k_folds[i]:
        progress += 1
        if(progress % 1000 == 0):
            print(progress)
        image_number = int(each_image_id[: each_image_id.find('_')])
        brain_id = each_image_id[each_image_id.find('_')+1:]

        case_path = os.path.join(DATASET_PATH,  brain_id)

        data_path = os.path.join(case_path, f'{ brain_id}_flair.nii');
        flair = nib.load(data_path).get_fdata()    

        data_path = os.path.join(case_path, f'{ brain_id}_t1ce.nii');
        ce = nib.load(data_path).get_fdata()

        data_path = os.path.join(case_path, f'{ brain_id}_t1.nii');
        t1 = nib.load(data_path).get_fdata()

        data_path = os.path.join(case_path, f'{ brain_id}_t2.nii');
        t2 = nib.load(data_path).get_fdata()

        data_path = os.path.join(case_path, f'{ brain_id}_seg.nii');
        seg = nib.load(data_path).get_fdata()
        
        X = np.zeros((192, 192, 4))
        Y = np.zeros((192, 192, 1))
        
        X[:, :, 0] = flair[ 30:222, 30:222, image_number ]
        X[:, :, 1] = ce[ 30:222, 30:222, image_number ]
        X[:, :, 2] = t1[ 30:222, 30:222, image_number ]
        X[:, :, 3] = t2[ 30:222, 30:222, image_number ]
        Y[:, :, 0] = seg[ 30:222, 30:222, image_number ]
        
        Y[Y==4] = 3
        fold_data_X.append(X)
        fold_data_Y.append(Y)
    fold_data_X = np.array(fold_data_X)
    fold_data_Y = np.array(fold_data_Y)
    
    print(fold_data_X.shape)
    print(fold_data_Y.shape)
    np.savez_compressed("../Dataset/folds_data/SKFold{}_X.npz".format(i), fold_data_X)
    np.savez_compressed("../Dataset/folds_data/SKFold{}_Y.npz".format(i), fold_data_Y)


5648
1000
2000
3000
4000
5000
(5648, 192, 192, 4)
(5648, 192, 192, 1)
5648
6000
7000
8000
9000
10000
11000
(5648, 192, 192, 4)
(5648, 192, 192, 1)
5648
12000
13000
14000
15000
16000
(5648, 192, 192, 4)
(5648, 192, 192, 1)
5648
17000
18000
19000
20000
21000
22000
(5648, 192, 192, 4)
(5648, 192, 192, 1)
5648
23000
24000
25000
26000
27000
28000
(5648, 192, 192, 4)
(5648, 192, 192, 1)


In [40]:
progress= 0

test_data_X = []
test_data_Y = []

print(len(test_brain_image_ids))

for each_image_id in test_brain_image_ids:
        progress += 1
        if(progress % 1000 == 0):
            print(progress)
        image_number = int(each_image_id[: each_image_id.find('_')])
        brain_id = each_image_id[each_image_id.find('_')+1:]

        case_path = os.path.join(DATASET_PATH,  brain_id)

        data_path = os.path.join(case_path, f'{ brain_id}_flair.nii');
        flair = nib.load(data_path).get_fdata()    

        data_path = os.path.join(case_path, f'{ brain_id}_t1ce.nii');
        ce = nib.load(data_path).get_fdata()

        data_path = os.path.join(case_path, f'{ brain_id}_t1.nii');
        t1 = nib.load(data_path).get_fdata()

        data_path = os.path.join(case_path, f'{ brain_id}_t2.nii');
        t2 = nib.load(data_path).get_fdata()

        data_path = os.path.join(case_path, f'{ brain_id}_seg.nii');
        seg = nib.load(data_path).get_fdata()
        
        X = np.zeros((192, 192, 4))
        Y = np.zeros((192, 192, 1))
        
        X[:, :, 0] = flair[ 30:222, 30:222, image_number ]
        X[:, :, 1] = ce[ 30:222, 30:222, image_number ]
        X[:, :, 2] = t1[ 30:222, 30:222, image_number ]
        X[:, :, 3] = t2[ 30:222, 30:222, image_number ]
        Y[:, :, 0] = seg[ 30:222, 30:222, image_number ]
        
        Y[Y==4] = 3
        test_data_X.append(X)
        test_data_Y.append(Y)
        
test_data_X = np.array(test_data_X)
test_data_Y = np.array(test_data_Y)
    
print(test_data_X.shape)
print(test_data_Y.shape)
np.savez_compressed("../Dataset/folds_data/TestData_X.npz", test_data_X)
np.savez_compressed("../Dataset/folds_data/TestData_Y.npz", test_data_Y)


7066
1000
2000
3000
4000
5000
6000
7000
(7066, 192, 192, 4)
(7066, 192, 192, 1)
